This script makes a **demo curriculum dataset**, saves it to Excel, and then loads it into a database (SQLite). The goal is to test how fast we can insert large amounts of data into related tables.

**Steps in the script:**

1. **Create Demo Data**

    - Makes rows with three columns: `strand`, `substrand`, and `activity`.
    - Strands are named `"Strand A"`, `"Strand B"`, etc.
    - Substrands are named `"Sub Strand A"`, `"Sub Strand B"`, etc.
    - Activities are named `"Activity #1"`, `"Activity #2"`, etc.
    - The number of activities per substrand is set by `ACTIVITIES_PER_SUBSTRAND`.
    - Saves everything into an Excel file called **`DEMO_CURRICULUM.xlsx`**.

2. **Set Up Database**

    - Uses SQLite and creates 3 tables:

        - **Strand** (`id`, `name`)
        - **Substrand** (`id`, `strand_id`, `name`)
        - **Activity** (`id`, `substrand_id`, `name`)

    - Each table is linked:

        - A substrand belongs to a strand.
        - An activity belongs to a substrand.

3. **Insert Data into Database**

    - Finds all unique strands and inserts them into the `Strand` table.

        - Keeps a dictionary `strand_id_map` to remember the IDs.

    - Finds all unique strand–substrand pairs and inserts them into the `Substrand` table.

        - Keeps a dictionary `substrand_id_map` for their IDs.

    - Inserts all activities, using the `substrand_id_map` to connect them to the right substrand.
    - Uses `executemany()` for batch inserts, which is much faster than inserting one row at a time.
    - Measures how long each step takes and prints the times.

4. **Result**

    - At the end, you get a SQLite database file called **`curriculum.db`** with all the data linked correctly.
    - The script also shows how many rows were inserted and how long it took.


In [8]:
import sqlite3
import string
import time
from pathlib import Path

import pandas as pd

# 1. CREATE DEMO EXCEL FILE


In [9]:
ACTIVITIES_PER_SUBSTRAND = 100

print("📂 Creating demo Excel file...")

start_time = time.perf_counter()

strands = list(string.ascii_uppercase)
sub_strands = list(string.ascii_uppercase)
activities = [str(a) for a in range(1, ACTIVITIES_PER_SUBSTRAND)]

all_rows = []
for strand in strands:
    for sub_strand in sub_strands:
        for activity in activities:
            all_rows.append(
                {
                    "strand": f"Strand {strand}",
                    "substrand": f"Sub Strand {sub_strand}",
                    "activity": f"Activity #{activity}",
                }
            )

df = pd.DataFrame(all_rows)

output_dir = Path("demos")
output_dir.mkdir(exist_ok=True)
output_path = output_dir / "DEMO_CURRICULUM.xlsx"
df.to_excel(output_path, index=False, engine="openpyxl")

elapsed = time.perf_counter() - start_time
print(f"✅ Excel created with {len(df)} rows in {elapsed:.2f} seconds.")
print(f"   File: {output_path.resolve()}")

📂 Creating demo Excel file...
✅ Excel created with 66924 rows in 4.96 seconds.
   File: /home/kraigochieng/projects/kicd_extraction/demos/DEMO_CURRICULUM.xlsx


# 2. DB INSERTION

## Create tables

In [10]:
print("\n📂 Importing into database...")

conn = sqlite3.connect("curriculum.db")
cur = conn.cursor()

cur.executescript("""
DROP TABLE IF EXISTS Strand;
DROP TABLE IF EXISTS Substrand;
DROP TABLE IF EXISTS Activity;

CREATE TABLE Strand (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
);

CREATE TABLE Substrand (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    strand_id INTEGER,
    name TEXT,
    UNIQUE(strand_id, name),
    FOREIGN KEY (strand_id) REFERENCES Strand(id)
);

CREATE TABLE Activity (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    substrand_id INTEGER,
    name TEXT,
    UNIQUE(substrand_id, name),
    FOREIGN KEY (substrand_id) REFERENCES Substrand(id)
);
""")




📂 Importing into database...


## Insert Strands

In [11]:
# --- Step 1: Insert Strands ---
start_time = time.perf_counter()
unique_strands = df["strand"].unique().tolist()
cur.executemany("INSERT INTO Strand (name) VALUES (?)", [(s,) for s in unique_strands])
strand_id_map = {
    s: i for s, i in zip(unique_strands, range(1, len(unique_strands) + 1))
}
elapsed = time.perf_counter() - start_time
print(f"✅ Inserted {len(unique_strands)} strands in {elapsed:.2f} seconds.")



✅ Inserted 26 strands in 0.02 seconds.


## Insert Substrands

In [12]:
# --- Step 2: Insert Substrands ---
start_time = time.perf_counter()
unique_pairs = df[["strand", "substrand"]].drop_duplicates().values.tolist()
cur.executemany(
    "INSERT INTO Substrand (strand_id, name) VALUES (?, ?)",
    [(strand_id_map[strand], substrand) for strand, substrand in unique_pairs],
)

substrand_id_map = {
    (strand, substrand): i
    for i, (strand, substrand) in enumerate(unique_pairs, start=1)
}
elapsed = time.perf_counter() - start_time
print(f"✅ Inserted {len(unique_pairs)} substrands in {elapsed:.2f} seconds.")

✅ Inserted 676 substrands in 0.05 seconds.


## Insert Activities

In [13]:
# --- Step 3: Insert Activities ---
start_time = time.perf_counter()
unique_triples = (
    df[["strand", "substrand", "activity"]].drop_duplicates().values.tolist()
)
cur.executemany(
    "INSERT INTO Activity (substrand_id, name) VALUES (?, ?)",
    [
        (substrand_id_map[(strand, substrand)], activity)
        for strand, substrand, activity in unique_triples
    ],
)
elapsed = time.perf_counter() - start_time
print(f"✅ Inserted {len(unique_triples)} activities in {elapsed:.2f} seconds.")


✅ Inserted 66924 activities in 0.49 seconds.


## Close Connection

In [14]:
# Commit
conn.commit()
conn.close()

print("\n🎉 Import complete!")


🎉 Import complete!
